해당 노트북은  [Deep-Learning-Paper-Review-and-Practice](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_LSTM_Tutorial.ipynb) 내용을 바탕으로 작성되었습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AIKU/Github/AIKU-DL-Paper-Review/code_practices/Seq2Seq

/content/drive/MyDrive/AIKU/Github/AIKU-DL-Paper-Review/code_practices/Seq2Seq


# Sequece to Sequnce Learning with Neural Networks (NIPS 2014) 실습

## BLEU Score 계산을 위한 라이브러리

In [ ]:
!pip install torchtext==0.6.0

In [5]:
import torchtext
print(torchtext.__version__)

0.6.0


## Preprocessing

In [ ]:
!python -m spacy download en # 영어 전처리 모듈 설치
!python -m spacy download de # 독일어 전처리 모듈 설치

### spaCy 라이브러리

In [7]:
import spacy

spacy_en=spacy.load('en_core_web_sm') # 영어 토큰화
spacy_de=spacy.load('de_core_news_sm') # 독일어 토큰화

In [8]:
# 토큰화 기능 써보기
tokenized=spacy_en.tokenizer("I am a graduate student.")

for i,token in enumerate(tokenized):
    print(f'index {i}: {token.text}')

index 0: I
index 1: am
index 2: a
index 3: graduate
index 4: student
index 5: .


In [21]:
# 토크나이저
from torchtext.data.utils import get_tokenizer

tokenize_en = get_tokenizer('spacy', language='en_core_web_sm') # 영어문장 토큰화
tokenize_de = get_tokenizer('spacy', language='de_core_news_sm') # 독어문장 토큰화

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


### field 라이브러리
+ Source(SRC) : de
+ Target(TRG) : en

In [22]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [23]:
# 영어-독일어 번역 데이터셋 불러오기
from torchtext.datasets import Multi30k

train_dataset = Multi30k(path='./dataset/multi30k/',exts=('train.de','train.en'), fields=(SRC,TRG))
valid_dataset = Multi30k(path='./dataset/multi30k/',exts=('val.de','val.en'), fields=(SRC,TRG))
test_dataset = Multi30k(path='./dataset/multi30k/',exts=('test_2016_flickr.de','test_2016_flickr.en'), fields=(SRC,TRG))

In [24]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [25]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [26]:
print(*train_dataset.examples[30].src,sep=' ')

ein mann , der mit einer tasse kaffee an einem urinal steht .


In [27]:
# build_vocab을 이용해 영어/독어 단어 사전 생성
## 최소 2번 이상 등장한 단어만 선택

SRC.build_vocab(train_dataset,min_freq=2)
TRG.build_vocab(train_dataset,min_freq=2)

print(f'len(SRC): {len(SRC.vocab)}')
print(f'len(TRG): {len(TRG.vocab)}')

len(SRC): 7853
len(TRG): 5893


In [55]:
## .stoi(string->int)를 통해서 단어 사전에서의
## 특정 단어와 맵핑된 고유한 정수를 출력
print(TRG.vocab.stoi['abcabc']) # <unk>=0 없는 단
print(TRG.vocab.stoi[TRG.pad_token]) # <pad>=1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


### iterator
+ 한 문장의 단어들이 순서대로 네트워크에 입력되어야 함
+ 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들기
    + 이를 위해 BucketIterator를 사

In [34]:
# 한 문장의 단어들이 순서대로 네트워크에 입력되어야 함
## 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들기
## 이를 위해 BucketIterator를 사용합니다.
## 배치 크기(batch size): 128
import torch

device=torch.device('cuda' if  torch.cuda.is_available() else 'cpu')

BATCH_SIZE=128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset,valid_dataset,test_dataset),
    batch_size=BATCH_SIZE,
    device=device,

)

In [59]:
for i,batch in enumerate(train_iterator):
    src = batch.src.permute(1,0)
    trg = batch.trg.permute(1,0)

    print(f'첫번째 배치 크기: {trg.shape}') # 단어개수 x 배치크기

    # 현재 배치의 첫번째 문장 출력
    for i in range(trg.shape[0]):
        word_num = trg[i][0].item()
        print(f'index {i}: {word_num} => {TRG.vocab.itos[word_num]}')
    break

첫번째 배치 크기: torch.Size([28, 128])
index 0: 2 => <sos>
index 1: 209 => this
index 2: 10 => is
index 3: 4 => a
index 4: 9 => man
index 5: 36 => standing
index 6: 8 => on
index 7: 4 => a
index 8: 2059 => rooftop
index 9: 13 => with
index 10: 216 => construction
index 11: 6 => in
index 12: 7 => the
index 13: 98 => background
index 14: 5 => .
index 15: 3 => <eos>
index 16: 1 => <pad>
index 17: 1 => <pad>
index 18: 1 => <pad>
index 19: 1 => <pad>
index 20: 1 => <pad>
index 21: 1 => <pad>
index 22: 1 => <pad>
index 23: 1 => <pad>
index 24: 1 => <pad>
index 25: 1 => <pad>
index 26: 1 => <pad>
index 27: 1 => <pad>


## 인코더(Encoder) 아키텍쳐
+ SRC 문장을 Context Vector로 인코딩
+ LSTM은 hidden state와 cell state를 반환
    + **input_dim**: 단어들을 One-Hot 인코딩할 때의 하나의 단어의 차원
    + **embed_dim**: 임베딩 차원
    + **hidden_dim**
    + **n_layers**: RNN 레이어 개수
    + **dropout_ratio**

In [61]:
import torch.nn as nn

In [67]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 단어 고유 정수값 -> embed_dim 차원으로 임베딩
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    # 소스문장(src)을 입력으로 받아 Context vector로 반환
    def forward(self, src):
        # src: [배치크기 x 단어개수], 각 단어의 고유 int 정보
        ## print(f'src.shape : {src.shape} -> ',end='')
        embedded = self.dropout(self.embedding(src))
        # embedded: [배치크기 x 단어개수 x 임베딩 차원]
        ## print(f'embedded.shape: {embedded.shape}')

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수 x 배치크기 x 히든차원], 현재 단어의 출력 정보
        # hidden: [레이어 개수 x 배치크기 x 히든차원], 현재까지의 모든 단어의 정보
        # cell: [레이어 개수 x 배치크기 x 히든차원], 현재까지의 모든 단어의 정보

        # Context vector 반환
        return hidden,cell


In [63]:
INPUT_DIM = len(SRC.vocab) # 7853
ENCODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5

In [65]:
enc_test = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS,ENC_DROPOUT_RATIO)
enc_test.to(device)

Encoder(
  (embedding): Embedding(7853, 256)
  (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [66]:
for i,batch in enumerate(train_iterator):
    src = batch.src.permute(1,0)

    hidden, cell = enc_test(src)
    print(f'hidden.shape: {hidden.shape}' )
    print(f'cell.shape: {cell.shape}' )

    break


src.shape : torch.Size([31, 128]) -> embedded.shape: torch.Size([31, 128, 256])
hidden.shape: torch.Size([2, 128, 512])
cell.shape: torch.Size([2, 128, 512])


## 디코더(Decoder) 아키텍처
+ Context vector를 타겟 문장으로 디코딩
+ LSTM은 hidden state와 cell state를 반환
    + **output_dim**: 단어에 대한 원핫 인코딩 차원
    + **embed_dim**: 임베딩 차원
    + **hidden_dim**: hidden_state 차원
    + **n_layers**: RNN 레이어의 개수
    + **dropout_ratio**: 드롭아웃 비율

In [68]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 한 단어의 원핫인코딩 차원 -> 특정차원으로 임베딩
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # FC 레이어 -> 인코더와 다른 부분
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, input, hidden, cell):
        # input: [배치크기], 단어의 개수는 항상 1개
        # hidden: [레이어개수 x 배치크기 x 히든차원]
        # cell: [레이어개수 x 배치크기 x 히든차원]
        input=input.unsqueeze(0)
        # [배치크기] -> [1 x 배치크기]

        embedded = self.dropout(self.embedding(input))

        ouput, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # outputs: [단어 개수=1 x 배치크기 x 히든차원], 현재 단어의 출력 정보
        # hidden: [레이어 개수 x 배치크기 x 히든차원], 현재까지의 모든 단어의 정보
        # cell: [레이어 개수 x 배치크기 x 히든차원], 현재까지의 모든 단어의 정보

        prediction = self.fc_out(ouput.squeeze(0))

        return prediction, hidden, cell

In [69]:
OUTPUT_DIM = len(TRG.vocab) # 5893
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
DEC_DROPOUT_RATIO = 0.5

In [78]:
dec_test = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS,DEC_DROPOUT_RATIO)
dec_test.to(device)

Decoder(
  (embedding): Embedding(5893, 256)
  (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  (fc_out): Linear(in_features=512, out_features=5893, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [83]:
print(outputs[:,0].shape)

torch.Size([30, 5893])


In [85]:
import random

for i,batch in enumerate(train_iterator):
    trg = batch.trg.permute(1,0)

    print(trg.shape)

    trg_len = trg.shape[0] # 단어 개수
    batch_size = trg.shape[1] # 배치 크기
    trg_vocab_size = dec_test.output_dim # 출력 차원
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
    # [단어개수 x 배치사이즈 x 출력차원] 공간만들기

    input = trg[0] # [배치사이즈] 첫 입력은 항상 <sos>

    print(f' 첫번째 문장\n {outputs[:,0]}({outputs[:,0].shape})')
    for t in range(1, trg_len):
        output, hidden, cell = dec_test(input, hidden, cell)

        outputs[t] = output
        print(f' {t}번째 단어 채운 후\n {outputs[:,0]}({outputs[:,0].shape})')

        top1 = output.argmax(1) # 가장 높은 확률인 단어의 인덱스 추출
        teacher_force = random.random() < 0.5
        input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기

    break


torch.Size([26, 128])
 첫번째 문장
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')(torch.Size([26, 5893]))
 1번째 단어 채운 후
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0350,  0.0233,  0.0275,  ...,  0.0378, -0.0077, -0.0199],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SelectBackward0>)(torch.Size([26, 5893]))
 2번째 단어 채운 후
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0350,  0.0233,  0.0275,  ...,  0.0378, -0.0077, -0.0199

## Seq2Seq 아키텍쳐
+ **인코더**: 주어진 소스문장을 context vector로 인코딩
+ **디코더**: 주어진context vector를 타겟 문장으로 디코딩
    + 한번에 하나의 단어씩 넣어서 구현

In [86]:
import random
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        hidden, cell= self.encoder(src)

        trg_len = trg.shape[0] # 단어개수
        batch_size = trg.shape[1] # 배치크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output
            top1 = output.argmax(1)

            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1

        return outputs

## Training

In [87]:
INPUT_DIM = len(SRC.vocab) # 7853
OUTPUT_DIM = len(TRG.vocab) # 5893
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [88]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

In [89]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights) # 가중치초기화

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [90]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [105]:
for i,batch in enumerate(train_iterator):
    src = batch.src.permute(1,0)
    trg = batch.trg.permute(1,0)

    print(trg[1:])
    print(trg[1:].reshape(-1))
    break

tensor([[   4,   16,   16,  ...,    4,   16,   14],
        [   9, 1227,   59,  ...,  354,   24,    6],
        [  22,   17,  282,  ...,  583,  127,   25],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
tensor([ 4, 16, 16,  ...,  1,  1,  1], device='cuda:0')


In [112]:
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습모드
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src.permute(1,0)
        trg = batch.trg.permute(1,0)

        optimizer.zero_grad()

        output = model(src, trg) # [단어개수 x 배치크기 x 출력차원]
        output_dim = output.shape[-1] # 출력차원 == trg vocab size

        output = output[1:] # 시작단어는 사용하지 않음
        output = output.view(-1, output_dim) # [ (단어개수-1) * 배치크기 x 출력차원] -> 예측값의 확률들
        trg = trg[1:].reshape(-1) # [ (단어개수-1) * 배치크기 ] -> 정답값

        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [113]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src.permute(1,0)
            trg = batch.trg.permute(1,0)

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]

            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].reshape(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [114]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 학습

In [115]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

In [116]:
# 학습 시
for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './pt/seq2seq.pt')

    print(f'Epoch: {epoch +1:02} | Time {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time 0m 32s
	Train Loss: 5.046 | Train PPL: 155.458
	 Val. Loss: 5.873 |  Val. PPL: 355.236
Epoch: 02 | Time 0m 31s
	Train Loss: 4.458 | Train PPL: 86.282
	 Val. Loss: 4.750 |  Val. PPL: 115.575
Epoch: 03 | Time 0m 32s
	Train Loss: 4.136 | Train PPL: 62.553
	 Val. Loss: 4.515 |  Val. PPL: 91.382
Epoch: 04 | Time 0m 32s
	Train Loss: 3.927 | Train PPL: 50.769
	 Val. Loss: 4.476 |  Val. PPL: 87.895
Epoch: 05 | Time 0m 33s
	Train Loss: 3.770 | Train PPL: 43.377
	 Val. Loss: 4.333 |  Val. PPL: 76.203
Epoch: 06 | Time 0m 31s
	Train Loss: 3.618 | Train PPL: 37.254
	 Val. Loss: 4.148 |  Val. PPL: 63.277
Epoch: 07 | Time 0m 32s
	Train Loss: 3.466 | Train PPL: 31.994
	 Val. Loss: 4.076 |  Val. PPL: 58.928
Epoch: 08 | Time 0m 31s
	Train Loss: 3.316 | Train PPL: 27.556
	 Val. Loss: 3.904 |  Val. PPL: 49.583
Epoch: 09 | Time 0m 32s
	Train Loss: 3.182 | Train PPL: 24.088
	 Val. Loss: 3.897 |  Val. PPL: 49.231
Epoch: 10 | Time 0m 31s
	Train Loss: 3.081 | Train PPL: 21.790
	 Val. Loss: 3.8

In [ ]:
# 모델 구조 저장
torch.save(model, './model/model.pth')

In [118]:
# 학습된 모델 로컬에 저장
from google.colab import files

files.download('./pt/seq2seq.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 모델 결과 확인

In [131]:
# 모델 구조 & 파라미터 가져오기
model2=torch.load('./model/model.pth')
model2.load_state_dict(torch.load('./pt/seq2seq.pt'))
model2.eval()

# test score
test_loss = evaluate(model2, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 3.694 | Test PPL: 40.218


### 번역해보기

In [138]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval()

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens =  [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [139]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', 'eine', 'mutter', 'und', 'ihr', 'kleiner', 'sohn', 'genießen', 'einen', 'schönen', 'tag', 'im', 'freien', '.', '<eos>']
소스 문장 인덱스: [2, 8, 364, 10, 134, 70, 624, 565, 19, 780, 200, 20, 88, 4, 3]
모델 출력 결과: a mother and her mother are enjoying a picture in a city . <eos>
